In [ ]:
# import sys
# sys.setrecursionlimit(100000)

In [1]:
!pip install gluonnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 5.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595740 sha256=0e63ad65ec8708c599e7e69b04af1f468b17e7ea21eb20b112de81242af239aa
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp


In [2]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.8 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-h0wxrowy
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-h0wxrowy
     |████████████████████████████████| 129 kB 5.2 MB/s 
     |████████████████████████████████| 54.7 MB 17 kB/s 
     |████████████████████████████████| 4.5 MB 46.6 MB/s 
     |████████████████████████████████| 1.2 MB 47.0 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:39tcmalloc: large alloc 1147494400 bytes == 0x399f4000 @  0x7f8dc5434615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 1

In [4]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch import optim
import numpy as np

In [5]:
import gluonnlp as nlp

In [6]:
from kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer

In [7]:
from tqdm import tqdm, tqdm_notebook

In [8]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [9]:
# GPU 설정
device = torch.device("cuda:0")

In [10]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [11]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [12]:
# 구글드라이브에 있는 파일 가져오기 위해
# 구글드라이브와 연동
# from google.colab import auth
# auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import pandas as pd

In [20]:
df = pd.read_excel('/content/drive/MyDrive/최종사전_증강(긍부정).xlsx')

In [24]:
df

word emotion  emo_num
0                           가능성이 늘어나다      만족        4
1                            가능성이 있다고      기쁨        1
2                                가능하다      편안        0
3                            가볍고 상쾌하다      기쁨        1
4                             가볍고 상쾌한      기쁨        1
...                               ...     ...      ...
28900       대신에 끝나면 얼마든지 귀여워해 줘도 된단다.      행복        2
28901  대신에 시 시험 끝나면 얼마든지 귀여워해 줘도 된단다.      행복        2
28902        열심히 저 네! 할게요! 형님, 감사합니다!      행복        2
28903        네! 형님, 열심히 할게요! 저 감사합니다!      행복        2
28904        네! 감사합니다! 열심히 할게요! 형님, 저      행복        2

[28905 rows x 3 columns]

In [25]:
df.sample(n=10)

word emotion  emo_num
23490         주위에 여자 회사 동기들 있는데 망했다 ㅠㅠ      걱정       16
16137   내일도 소마는 야구장에서 박용택 선수님 응원합니다 ^^      기쁨        1
14654               역시...장꾸준!대박입니당!^^♡      만족        4
3851                             몹시 야윈      걱정       16
510                           단정하게 포개다      편안        0
20286     나오늘 전달받은거없는데 퇴근시간20분넘고불안하다..      불안       10
20404     근데 정보가 많진 않으니 선뜻 지원하기가 쉽지않네요      답답       12
4231                          방해하거나 막다      불만       17
24271                        아 불가능한가요?      걱정       16
24959  두산 장원진선수의 오바로인해 주눅들지 않을까 걱정됩니다.      걱정       16

In [26]:
df['emo_num'].unique()

array([ 4,  1,  0,  2,  5,  3, 10, 11,  7,  9, 13,  6,  8, 17, 15, 16, 12,
       14, 18, 19, 20])

In [27]:
data_list =[]
for i,j in zip(df['word'], df['emo_num']):
  data = []
  data.append(i)
  data.append(str(j))

  data_list.append(data)


In [28]:
print(data_list[0])

['가능성이 늘어나다', '4']


In [29]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [30]:
print(len(dataset_train))
print(len(dataset_test))

21678
7227


In [31]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [32]:
# Setting parameters
max_len = 56 
batch_size = 50
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [33]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [34]:
data_train[0]

(array([   2, 4924, 7396, 7086, 3817,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1], dtype=int32),
 array(6, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 11)

In [35]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [36]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=21,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# !pip install sentencepiece==0.1.96

In [37]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [38]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/434 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.0558059215545654 train acc 0.06
epoch 1 batch id 201 loss 2.6118597984313965 train acc 0.11014925373134327
epoch 1 batch id 401 loss 2.1171953678131104 train acc 0.18872817955112217
epoch 1 train acc 0.20103686635944695


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/145 [00:00<?, ?it/s]

...

epoch 6 test acc 0.6320664112388248


  0%|          | 0/434 [00:00<?, ?it/s]

epoch 7 batch id 1 loss 0.4641125500202179 train acc 0.86
epoch 7 batch id 201 loss 0.4455232322216034 train acc 0.8881592039800994
epoch 7 batch id 401 loss 0.345115065574646 train acc 0.8885286783042384
epoch 7 train acc 0.8888874259381159


  0%|          | 0/145 [00:00<?, ?it/s]

epoch 7 test acc 0.6388045977011492


In [39]:
#모델 저장하기
torch.save(model,'/content/drive/MyDrive/last_final_model.pt')

In [ ]:
#!pip install torchinfo

In [ ]:
# from torchinfo import summary
# model2 = model.eval()
# summary(model2)

In [40]:
#토큰화
# tokenizer = get_tokenizer()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("편안함이")
            elif np.argmax(logits) == 1:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 2:
                test_eval.append("행복이")
            elif np.argmax(logits) == 3:
                test_eval.append("감사가")
            elif np.argmax(logits) == 4:
                test_eval.append("만족이")
            elif np.argmax(logits) == 5:
                test_eval.append("흥미로움이")
            elif np.argmax(logits) == 6:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 7:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 8:
                test_eval.append("당황이")
            elif np.argmax(logits) == 9:
                test_eval.append("상처가")
            elif np.argmax(logits) == 10:
                test_eval.append("불안이")
            elif np.argmax(logits) == 11:
                test_eval.append("짜증이")
            elif np.argmax(logits) == 12:
                test_eval.append("답답이")
            elif np.argmax(logits) == 13:
                test_eval.append("허무함이")
            elif np.argmax(logits) == 14:
                test_eval.append("조롱이")
            elif np.argmax(logits) == 15:
                test_eval.append("불편함이")
            elif np.argmax(logits) == 16:
                test_eval.append("걱정이")
            elif np.argmax(logits) == 17:
                test_eval.append("불만이")
            elif np.argmax(logits) == 18:
                test_eval.append("무서움이")
            elif np.argmax(logits) == 19:
                test_eval.append("보통이")
            else:
                test_eval.append("궁금이")


        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [41]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0':
      break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 나는 괴롭힘을 당한다
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 여전히 기쁘구나
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 0


In [ ]:
predict('저는 진짜 모르겠어요')